# Ejemplo de código de optimizador de prompts
Este código muestra cómo optimizar un prompt para un modelo de lenguaje GPT-3.5. El codigo esta basado en el paper [Large Language Models as Optimizers. Yang et al. (2023).](https://doi.org/10.48550/arXiv.2309.03409)


## 1.- Setup inicial

### 1.1- Instalar librerías

In [ ]:
#! pip install openai
#! pip install tenacity
#! pip install openai[datalib]
#! pip install python-dotenv
#! pip install tabulate

### 1.2.- Cargar librerías

In [ ]:
import openai
import os
from dotenv import load_dotenv
from tabulate import tabulate
from tenacity import retry, wait_random_exponential, stop_after_attempt

### 1.3.- Variables de entorno

In [ ]:
# Load secrets and config from .env file
load_dotenv()

# OpenAI API
openai.api_type = os.getenv("OPENAI_API_TYPE")
openai.api_base = os.getenv("OPENAI_API_BASE")
openai.api_version = os.getenv("OPENAI_API_VERSION")
openai.api_key = os.getenv("OPENAI_API_KEY")
embedding_model = os.getenv("OPENAI_EMBEDDING_MODEL")
print("OpenAI API key: {}".format(openai.api_key[:5] + '...' + openai.api_key[-5:]))
print("OpenAI API base: {}".format(openai.api_base))
print("OpenAI API version: {}".format(openai.api_version))
print("OpenAI API type: {}".format(openai.api_type))

# Model endpoint names
gpt35_model = os.getenv("OPENAI_GPT35_MODEL")
gpt35_16k_model = os.getenv("OPENAI_GPT35_16K_MODEL")
gpt4_model = os.getenv("OPENAI_GPT4_MODEL")
print("GPT-3.5-Turbo model: {}".format(gpt35_model))
print("GPT-3.5-Turbo-16k model: {}".format(gpt35_16k_model))
print("GPT-4 model: {}".format(gpt4_model))

### 1.4.- Clase para creación de embeddings (vectores)

In [ ]:
@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(5))
def generate_embeddings(text):
    response = openai.Embedding.create(input=text, engine=embedding_model)
    embeddings = response["data"][0]['embedding']
    return embeddings


### 1.5.- Clase de interfaz con modelos GPT

In [ ]:
@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(3))
def generate_text(prompt, model=gpt4_model, messages=[], max_tokens=600, temperature=0.5, top_p=1.0, frequency_penalty=0.0, presence_penalty=0.0, stop=None):
    _messages = []
    _messages.extend(messages)
    _messages.append({"role": "user", "content": prompt})
    
    print("\n\n============================ PROMPT ============================\n")
    for message in _messages:
        print(f"{message['role']}: {message['content']}")
        
    response = openai.ChatCompletion.create(
        engine=model,
        messages=_messages,
        max_tokens=max_tokens,
        temperature=temperature,
        top_p=top_p,
        frequency_penalty=frequency_penalty,
        presence_penalty=presence_penalty,
        stop=stop
    )
    print("\n\n============================ RESPONSE ============================\n")
    print(response["choices"][0]["message"]["content"])
    return response["choices"][0]["message"]["content"]

## 2.- 

In [ ]:
def generate_solution(instructions, problems):
    instructions_str = "\n".join([f"text:\n{i['prompt']}\nscore:\n{i['score']}" for i in instructions])
    
    problems_str = "\n".join([f"Q:\n{i['question']}\nA:<INS>\nGround Truth Answer:\n{i['ground_truth']}" for i in problems])
    
    prompt = f"Your task is to generate the instruction <INS>. Below are some previous instructions with their scores. The score ranges from 0 to 100.\n\n{instructions_str}\n\nBelow are some problems.\n{problems_str}\n\nGenerate an instruction that is different from all the instructions <INS> above, and has a higher score than all the instructions <INS> above. The instruction should begin with <INS> and end with </INS>. The instruction should be concise, effective, and generally applicable to all problems above."
    
    solution = generate_text(prompt, model=gpt4_model, temperature=1)
    solution = solution.replace("<INS>", "").replace("</INS>", "")
    return solution

In [ ]:
def evaluate_answer(answer, question, ground_truth):
    prompt = f"Q:\n{question}\nA:\n{answer}\nGround Truth Answer:\n{ground_truth}\n\nBased on the question, the answer and the ground truth answer: Give a score from 0 to 100 to the answer, where 0 means the answer is completely wrong, and 100 means the answer is completely correct.\nScore:"
    score = generate_text(prompt, model=gpt4_model, temperature=0.2)
    return int(score)

In [ ]:
def test_solution(solution, problems):
    scores = []
    for problem in problems:
        prompt = f"Q:{problem['question']}\nA:{solution}"
        answer = generate_text(prompt, model=gpt35_model, temperature=0.7)
        score = evaluate_answer(answer, problem['question'], problem['ground_truth'])
        scores.append(score)
    
    # return the average score
    return sum(scores) / len(scores)
    

In [ ]:
instructions = [
    {
        "prompt": "Let’s figure it out!",
        "score": 61
    },
    {
        "prompt": "Let’s solve the problem.",
        "score": 63
    }
]

problems = [
    {
        "question": "Alannah, Beatrix, and Queen are preparing for the new school year and have been given books by their parents. Alannah has 20 more books than Beatrix. Queen has 1/5 times more books than Alannah. If Beatrix has 30 books, how many books do the three have together?",
        "ground_truth": 140
    }
]

In [ ]:
MAX_ITERATIONS = 3

for i in range(MAX_ITERATIONS):
    solution = generate_solution(instructions, problems)
    score = test_solution(solution, problems)
    instructions.append({"prompt": solution, "score": score})
    
# use tabulate to print the results
print(tabulate(instructions, headers="keys"))
    